# Global Terrorism Attacks - Predicting The Responsible Group

**Table of Contents**
<div id="toc"></div>

## Load data

In [15]:
%matplotlib inline

import ConfigParser
config = ConfigParser.ConfigParser()
config.read('config.ini')
gtdConvertedFilename = 'gtd_converted.csv'

import pandas as pd
import csv
from sklearn import preprocessing
import os.path

if not os.path.isfile(gtdConvertedFilename):
    %run CreateCSVFile.py

gtd = pd.read_csv(gtdConvertedFilename, encoding='latin1', low_memory=False)
gtd.tail(3)

,Unnamed: 0,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
170347,170347,201612310043,2016,12,31,NaN,1,NaN,229,Democratic Republic of the Congo,...,NaN,"""DRC Armed Forces Dislodge Ugandan LRA Rebels ...",NaN,NaN,START Primary Collection,1,1,0,1,NaN
170348,170348,201612310044,2016,12,31,NaN,0,NaN,130,Mexico,...,There is doubt that this incident meets terror...,"""Gunmen shoot at home of Mexican indigenous ri...","""â Gunmen shoot at home of Mexican indigenou...",NaN,START Primary Collection,-9,-9,0,-9,NaN
170349,170349,201701270001,2016,12,30,NaN,0,NaN,195,Sudan,...,There is doubt that this incident meets terror...,"""Rebels To Blame For Nierteti Killing: Central...","""Security deteriorates in West Darfur alongsid...",NaN,START Primary Collection,0,0,0,0,NaN


## Preprocessing

### Filter Groups

In [16]:
from collections import Counter

group_count = Counter(gtd['gname'])
# Remove groups with 3 attacks or less
filtered_groups = [group for group, counter in group_count.items() if counter > 3]
# Remove 'Unknown'
filtered_groups.remove('Unknown')

gtd = gtd[gtd['gname'].isin(filtered_groups)]

gtd.shape

(88657, 136)

### Define Features

In [17]:
if config.getboolean('Booleans', 'UseLessFeatures') == True:
    columns_to_keep = ['gname', 'iyear', 'country', 'region', 'attacktype1', 'weaptype1']
else:
    columns_to_keep = ['gname', 'iyear', 'country', 'crit1', 'crit2', 'crit3', 'attacktype1', 'targtype1',
                       'targsubtype1','weaptype1', 'weapsubtype1', 'ransom']
    
gtd = gtd[columns_to_keep]
gtd.tail(3)

,gname,iyear,country,region,attacktype1,weaptype1
170345,Boko Haram,2016,146,11,9,13
170347,Lord's Resistance Army (LRA),2016,229,11,6,13
170349,Sudan Liberation Movement,2016,195,11,2,5


### Transform Target

In [18]:
number = preprocessing.LabelEncoder()
gtd['gname'] = number.fit_transform(gtd.gname) #number.inverse_transform()
gtd = gtd.rename(columns={'gname': 'groupId'})

### Fill NaNs

In [19]:
gtd=gtd.fillna(0) #TODO: 999 or 0?

gtd.tail(3)

,groupId,iyear,country,region,attacktype1,weaptype1
170345,214,2016,146,11,9,13
170347,555,2016,229,11,6,13
170349,919,2016,195,11,2,5


In [ ]:
gtd.to_csv('gtd_processed_11features.csv', encoding='utf-8', index=False)